# Linear Programming

Linear programming problems are a subset of optimisation problems that can be formulated as a set of linear constraints and a linear objective function.

$ \max_x z = \bf{c}^T \bf{x}$

$ A\bf{x} \geq b $

$ \bf{x} \geq 0 $

WHAT SPACE ARE THE VARIABLES IN?

The name "programming" comes from the older use of the word, meaning planning or scheduling, as the problem formulation and its treatment were developed in a military context when thinking about resource allocation [link].

LP problems are "easy" - we know if a solution exists it must exist somewhere on the constraint boundary. We can use the simplex method [link] implemented in many libraries to find a solution. Here we iterate around the vertices of the feasible region and look for an improvement in the objective function.

In high-dimensional linear programs, the simplex method can suffer from a "curse of dimensionality" problem - the time complexity to find a solution can be exponential with the number of constraints/variables. Other methods, referred to as "interior point" methods take a different approach:

- start the optimisation from somewhere in the feasible region 
- step in the direction of the the optimum

Step size and direction are set carefully - see the appendix on "dual problems" for more information.

## A "piece-of-cake" example LP problem

I've borrowed this example problem from the Coursera course xxxx

Suppose we have three friends, Alice, Bob and Charlie, looking to share a cake. They each want some, and they have a few rules about how they want to share it out:

- Alice wants at least as much as Bob
- Bob wants at least twice as much as Charlie
- Charlie wants at least 1/10th of the cake
- Bob's doctor says he can't have more than half of the cake

They'll give an amount to charity proportional to their fraction of the cake. Alice will give a fraction of £10, Bob will give a fraction of £12, and Charlie will give a fraction of £15


WRITING OUT THE PROBLEM


In [12]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, maximize, value

In [17]:
# we use the ConcreteModel object because we want to directly define variables and constraints in code
model = ConcreteModel()

# setting the variables
model.alice = Var(bounds=(0,1))
model.bob = Var(bounds=(0,1))
model.charlie = Var(bounds=(0,1))

# setting the objective function
def obj_function(model):
    return 10*model.alice + 12*model.bob + 15*model.charlie
model.objective = Objective(rule=obj_function, sense=maximize)

# setting the constraints CHANGE ME
model.alice_vs_bob= Constraint(rule=lambda model: model.alice >= model.bob)
model.bob_vs_charlie = Constraint(rule=lambda model: model.bob >= 2*model.charlie)
model.charlie_1_10 = Constraint(rule=lambda model: model.charlie >= 0.1)
model.bob_vs_doctor = Constraint(rule=lambda model: model.bob <= 0.5)
model.sum_rule = Constraint(rule=lambda model: model.alice + model.bob + model.charlie == 1)

# instantiating and calling the solver
solver = SolverFactory('glpk')
results = solver.solve(model)

# Check the solver status and whether the problem is feasible
solver_status = results.solver.status
solver_termination_condition = results.solver.termination_condition

# Output solver information
print(f"Solver Status: {solver_status}")
print(f"Termination Condition: {solver_termination_condition}")

print(f'Alice: {value(model.alice)}')
print(f'Bob: {value(model.bob)}')
print(f'Charlie: {value(model.charlie)}')

print(f'Money to charity: £{value(obj_function(model))}')


Solver Status: ok
Termination Condition: optimal
Alice: 0.4
Bob: 0.4
Charlie: 0.2
Money to charity: £11.8


# (Mixed) Integer Linear Programming

Integer Linear Programming problems (ILP) are LPs whose variables are constrained to integer space. Perhaps we are considering the number of vehicles to use on a delivery job, the number of batches of a product to purchase, etc.

Mixed Integer Linear Programming problems are LPs where some of the variables are constrained to integer space, but not all.

It's possible to encode a "decision" in a (M)ILP problem by defining a variable that must take a value in the set {0,1} - we can still use the same solvers to find optima for these problems.

Compared to LP problems, ILP problems are difficult. We might think that we could just solve the LP problem that comes from using the same constraints and objective function, but allowing the variables to be real-valued (the so-called "LP relaxation") and round the answer to integer space where necessary? But there's no guarantee this will meet the constraints or be optimal after the rounding. 

 The strategy for solving is:
- solve the LP relaxation
- use a variety of techniques to step in from the LP solution towards an integer solution

Here's an example ILP problem for illustration.

Suppose a company is trying to work out how many of each of two potential products they should make each week in order to make the most profit.

Product A returns a profit of £30 while product B returns a profit of £40.

The following rules are in place:
- A maximum of 30 B can be sold each week, due to regulatory requirements
- A and B share a component - Each A consumes 3 and each B consumes 1. The factory can only acquire 100 per week.
- A and B require time from one of 7 specialised members of staff who have 245 total hours available per week. Each A requires 4 hours and B requires 6 hours.

Writing this in notation:

Let $x_A$ be the number of product A produced, and $x_B$ be the number of product B. 

$\max 30x_A + 40x_B$

$x_A, x_B \isin \mathbb{Z}$

$s.t.$


$x_B \leq 30$

$3x_A + x_B \leq 100$

$4x_A + 6x_B \leq 245$



In [19]:
from pyomo.environ import NonNegativeIntegers

In [27]:
# Create a Concrete Model
model = ConcreteModel()

# Define decision variables (non-negative integers)
model.x1 = Var(domain=NonNegativeIntegers)
model.x2 = Var(domain=NonNegativeIntegers)

# Objective Function: Maximize the profit
model.profit = Objective(expr=30*model.x1 + 40*model.x2, sense=maximize)

# Constraints
# 1. Raw Material Constraint: 10x1 + 20x2 <= 400
model.material_constraint = Constraint(expr=3*model.x1 + model.x2 <= 100)

# 2. Labor Hours Constraint: 2x1 + 3x2 <= 180
model.labor_constraint = Constraint(expr=4*model.x1 + 6*model.x2 <= 245)

# 3. Machine Hours Constraint: 4x1 + 6x2 <= 240
model.machine_constraint = Constraint(expr=model.x2 <= 30)

# Create a solver
solver = SolverFactory('glpk')  

# Solve the model
result = solver.solve(model)

solver_status = results.solver.status
solver_termination_condition = results.solver.termination_condition

print(solver_status)
print(solver_termination_condition)
# Display results
model.display()

ok
optimal
Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  25.0 :  None : False : False : NonNegativeIntegers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  24.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1710.0

  Constraints:
    material_constraint : Size=1
        Key  : Lower : Body : Upper
        None :  None : 99.0 : 100.0
    labor_constraint : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 244.0 : 245.0
    machine_constraint : Size=1
        Key  : Lower : Body : Upper
        None :  None : 24.0 :  30.0


## A battery optimisation example

Finally, let's explore a more involved example where we optimise over a time index too, and use the "decision" methodology discussed above:

Problem Objective:
The goal of this MILP problem is to minimize the total energy cost of operating a battery over a day, while buying and selling the energy it stores. The battery can store energy, discharge it during peak price periods, and charge it during off-peak periods.

### Decision Variables:
- **$P_{\text{charge},t}$**: Continuous variable representing the power input (charging) to the battery at time $t$ (in kWh).
- **$P_{\text{discharge},t}$**: Continuous variable representing the power output (discharging) from the battery at time $t$ (in kWh).
- **$E_{t}$**: Continuous variable representing the energy stored in the battery at time $t$ (in kWh).
- **$u_{\text{charge},t}$**: Binary variable (0 or 1) indicating whether the battery is charging at time $t$.
- **$u_{\text{discharge},t}$**: Binary variable (0 or 1) indicating whether the battery is discharging at time $t$.
- **$P_{\text{grid},t}$**: Continuous variable representing the net power exchanged with the grid (can be positive or negative depending on buying or selling).

### Parameters:
- **$\eta_{\text{charge}}$**: Charging efficiency of the battery.
- **$\eta_{\text{discharge}}$**: Discharging efficiency of the battery.
- **$P_{\text{max}}$**: Maximum charging or discharging power of the battery (kW).
- **$E_{\text{max}}$**: Maximum energy capacity of the battery (kWh).
- **$E_{\text{min}}$**: Minimum energy capacity of the battery (kWh), usually 0.
- **$\text{Price}_{t}$**: Electricity price at time $t$ (price per kWh).
- **$T$**: Total number of time periods - in our case, 24 hours, but we can change the granularity as appropriate.


### Objective Function:
The objective is to minimize the total cost over the time horizon:

$
\text{Minimize} \sum_{t=1}^{T} \left( P_{\text{grid},t} \cdot \text{Price}_t \right)
$

Where $P_{\text{grid},t}$ is the net power drawn from (or sold to) the grid at time $t$, and $\text{Price}_t$ is the price of electricity at time $t$. This is minimized by charging the battery when prices are low and discharging when prices are high.

### Constraints:
**Energy Balance:** The energy stored in the battery at time $t$ is governed by the charging and discharging:

- $E_{t} = E_{t-1} + \eta_{\text{charge}} \cdot P_{\text{charge},t} - \frac{P_{\text{discharge},t}}{\eta_{\text{discharge}}}$

**Battery Capacity:** The energy stored in the battery must be within its capacity limits:

- $E_{\text{min}} \leq E_{t} \leq E_{\text{max}} \quad \forall t$

**Power Limits:** The charging and discharging power are limited by the maximum power rating of the battery:

- $0 \leq P_{\text{charge},t} \leq u_{\text{charge},t} \cdot P_{\text{max}} \quad \forall t$

- $0 \leq P_{\text{discharge},t} \leq u_{\text{discharge},t} \cdot P_{\text{max}} \quad \forall t$

**Mutual Exclusivity of Charging and Discharging:** The battery cannot charge and discharge at the same time:

- $u_{\text{charge},t} + u_{\text{discharge},t} \leq 1 \quad \forall t$

**Net Power to/from the Grid:** The net power exchanged with the grid is the difference between the demand and the battery’s discharge/charge:

- $P_{\text{grid},t} = P_{\text{load},t} - P_{\text{discharge},t} + P_{\text{charge},t} \quad \forall t$ 

where $P_{\text{load},t}$ is the non-grid load or demand at time $t$. This can be set to 0 if we're only concerned with energy arbitrage.

Let's write the model in pyomo:


In [38]:
import pyomo.environ as pyo

# Create a Pyomo ConcreteModel
model = pyo.ConcreteModel()

# Sets
T = 24  # Time periods (for a day with hourly intervals)
model.T = pyo.RangeSet(0, T-1)

# Parameters
eta_charge = 0.95  # Charging efficiency
eta_discharge = 0.9  # Discharging efficiency
P_max = 50  # Maximum charging/discharging power (kW)
E_max = 200  # Maximum energy capacity (kWh)
E_min = 0  # Minimum energy capacity
Price = [20 + 5 * (i // 6) for i in range(T)]  # Price profile (varies every 6 hours)
P_load = [30 + 10 * (i % 6) for i in range(T)]  # Load profile (varies cyclically)

# Variables
model.P_charge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, P_max))  # Charging power
model.P_discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, P_max))  # Discharging power
model.E = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(E_min, E_max))  # Energy stored in the battery
model.u_charge = pyo.Var(model.T, within=pyo.Binary)  # Charging decision
model.u_discharge = pyo.Var(model.T, within=pyo.Binary)  # Discharging decision
model.P_grid = pyo.Var(model.T, within=pyo.Reals)  # Net power to/from the grid

# Objective: Minimize total cost (buying from the grid or selling to the grid)
def objective_function(model):
    return sum(model.P_grid[t] * Price[t] for t in model.T)

model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Constraints

# Energy balance: E_t = E_{t-1} + eta_charge * P_charge,t - P_discharge,t / eta_discharge
def energy_balance(model, t):
    if t == 0:
        return model.E[t] == 0  # Assume the battery starts empty
    else:
        return model.E[t] == model.E[t-1] + eta_charge * model.P_charge[t] - model.P_discharge[t] / eta_discharge

model.energy_balance = pyo.Constraint(model.T, rule=energy_balance)

# Power limits for charging: 0 <= P_charge,t <= u_charge,t * P_max
def charge_limit(model, t):
    return model.P_charge[t] <= model.u_charge[t] * P_max

model.charge_limit = pyo.Constraint(model.T, rule=charge_limit)

# Power limits for discharging: 0 <= P_discharge,t <= u_discharge,t * P_max
def discharge_limit(model, t):
    return model.P_discharge[t] <= model.u_discharge[t] * P_max

model.discharge_limit = pyo.Constraint(model.T, rule=discharge_limit)

# Mutual exclusivity: u_charge,t + u_discharge,t <= 1
def mutual_exclusivity(model, t):
    return model.u_charge[t] + model.u_discharge[t] <= 1

model.mutual_exclusivity = pyo.Constraint(model.T, rule=mutual_exclusivity)

# Net power exchanged with the grid: P_grid,t = P_load,t - P_discharge,t + P_charge,t
def grid_power_balance(model, t):
    return model.P_grid[t] == P_load[t] - model.P_discharge[t] + model.P_charge[t]

model.grid_power_balance = pyo.Constraint(model.T, rule=grid_power_balance)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Display results
for t in model.T:
    print(f"Time {t}: P_charge = {pyo.value(model.P_charge[t]):.2f} kW, P_discharge = {pyo.value(model.P_discharge[t]):.2f} kW, E = {pyo.value(model.E[t]):.2f} kWh, P_grid = {pyo.value(model.P_grid[t]):.2f} kW, u_charge = {int(pyo.value(model.u_charge[t]))}, u_discharge = {int(pyo.value(model.u_discharge[t]))}")

print(f"Optimal Objective Function Value: £{pyo.value(model.obj):.2f}")



Time 0: P_charge = 0.00 kW, P_discharge = 50.00 kW, E = 0.00 kWh, P_grid = -20.00 kW, u_charge = 0, u_discharge = 1
Time 1: P_charge = 10.53 kW, P_discharge = 0.00 kW, E = 10.00 kWh, P_grid = 50.53 kW, u_charge = 1, u_discharge = 0
Time 2: P_charge = 50.00 kW, P_discharge = 0.00 kW, E = 57.50 kWh, P_grid = 100.00 kW, u_charge = 1, u_discharge = 0
Time 3: P_charge = 50.00 kW, P_discharge = 0.00 kW, E = 105.00 kWh, P_grid = 110.00 kW, u_charge = 1, u_discharge = 0
Time 4: P_charge = 50.00 kW, P_discharge = 0.00 kW, E = 152.50 kWh, P_grid = 120.00 kW, u_charge = 1, u_discharge = 0
Time 5: P_charge = 50.00 kW, P_discharge = 0.00 kW, E = 200.00 kWh, P_grid = 130.00 kW, u_charge = 1, u_discharge = 0
Time 6: P_charge = 0.00 kW, P_discharge = 0.00 kW, E = 200.00 kWh, P_grid = 30.00 kW, u_charge = 0, u_discharge = 0
Time 7: P_charge = 0.00 kW, P_discharge = 0.00 kW, E = 200.00 kWh, P_grid = 40.00 kW, u_charge = 0, u_discharge = 0
Time 8: P_charge = 0.00 kW, P_discharge = 0.00 kW, E = 200.00 kWh